In [1]:
from langdetect import detect
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
df_test = pd.read_csv("../data/qald-8-test.csv")

In [3]:
df_test=df_test.dropna(subset=['questions'])

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(df_test['questions'])

from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
X = tfidf_transformer.fit_transform(X_train)

# get feature names
feature_names=vectorizer.get_feature_names()

#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
def ExtractKeyWords(text):
    #generate tf-idf for the given text
    tf_idf_vector=tfidf_transformer.transform(text)
    
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    #extract only the top n; n here is 3
    keywords=extract_topn_from_vector(feature_names,sorted_items,3)
 
    # now print the results
    sentence = ""
    for k in keywords:
        sentence+=(" "+k)
    return sentence

new_set = []
for text in X:
    new_set.append(ExtractKeyWords(text))
    
df_test["questions"]=new_set

In [5]:
df_en = df_test[df_test["lang"]=='en']
df_fr = df_test[df_test["lang"]=='fr']
df_de = df_test[df_test["lang"]=='de']

In [6]:
import datetime

def test(data):
    f_prediction = []
    for sentence in data['questions']:
        f_prediction.append(detect(sentence))
    return f_prediction

In [7]:
a = datetime.datetime.now()
re_en = test(df_en)
b = datetime.datetime.now()
c = b-a
c/len(df_en)

datetime.timedelta(microseconds=18923)

In [8]:
a = datetime.datetime.now()
re_fr = test(df_fr)
b = datetime.datetime.now()
c = b-a
c/len(df_fr)

ZeroDivisionError: integer division or modulo by zero

In [ ]:
a = datetime.datetime.now()
re_de = test(df_de)
b = datetime.datetime.now()
c = b-a
c/len(df_de)

In [ ]:
df_test.lang.unique() 

In [ ]:
# EN
# q3 0.6565656565656566
# q4 0.7
# q5 0.6610169491525424
# q6 0.68
# q7 0.627906976744186
# q8 0.6097560975609756

In [ ]:
# FR
# q3 0.8585858585858586
# q4 0.88
# q5 0.9
# q6 0.83
# q7 0.9767441860465116
# q8

In [ ]:
# DE
# q3 0.6565656565656566
# q4 0.82
# q5 0.6938775510204082
# q6 0.62
# q7 0.627906976744186
# q8

In [9]:
accuracy_score(df_en['lang'], re_en)

0.6097560975609756

In [ ]:
accuracy_score(df_fr['lang'], re_fr)

In [ ]:
accuracy_score(df_de['lang'], re_de)